In [22]:
import secret_keys as key
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import pandas as pd

In [14]:
import tweepy

# Creating the authentication object
auth = tweepy.OAuthHandler(key.consumer_key, key.consumer_secret)
# Setting your access token and secret
auth.set_access_token(key.access_token, key.access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth,wait_on_rate_limit=True) 

In [6]:
search_words = '#corona OR #covid-19 OR #covid OR #flattenthecurve'
date_since = '2020-03-21'

In [7]:
tweets = tweepy.Cursor(api.search ,
                       q = search_words,
                       lang = 'en',
                       since = date_since
                      ).items(2000)

In [8]:
tweet_list = [[ tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location ] for tweet in tweets]

In [9]:
df_tweets = pd.DataFrame(tweet_list, columns = ['geo', 'tweet' , 'user_handle','location'])

In [10]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
geo            0 non-null object
tweet          2000 non-null object
user_handle    2000 non-null object
location       2000 non-null object
dtypes: object(4)
memory usage: 62.6+ KB


In [11]:
# https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90
# Tweet data cleaning
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [23]:
df_tweets['cleaned_tweets'] = df_tweets.tweet.progress_apply(tweet_cleaner)

In [24]:
df_tweets.head()

,geo,tweet,user_handle,location,cleaned_tweets
0,None,RT @YWCAUSA: Let your legislator know how impo...,DocantoLicy,"Washington, DC",rt let your legislator know how important nonp...
1,None,RT @TristanCorkPost: Other shoppers 'stormed' ...,dyan_deen,,rt other shoppers stormed into tesco during th...
2,None,RT @vtdigger: Substance abuse treatment provid...,suptmoran,,rt substance abuse treatment providers and rec...
3,None,RT @ACLU: Congress must act now to ensure the ...,JenPetterson,Planet 🌍 Earth,rt congress must act now to ensure the safety ...
4,None,RT @francesleach_: Cuban doctors arriving into...,afro_rriquena,"Manhattan, NY",rt cuban doctors arriving into italy to suppor...


In [25]:
df_tweets['cleaned_tweets'] = df_tweets.cleaned_tweets.str.replace('rt', '')

In [26]:
df_tweets.head()

,geo,tweet,user_handle,location,cleaned_tweets
0,None,RT @YWCAUSA: Let your legislator know how impo...,DocantoLicy,"Washington, DC",let your legislator know how impoant nonprofi...
1,None,RT @TristanCorkPost: Other shoppers 'stormed' ...,dyan_deen,,other shoppers stormed into tesco during the ...
2,None,RT @vtdigger: Substance abuse treatment provid...,suptmoran,,substance abuse treatment providers and recov...
3,None,RT @ACLU: Congress must act now to ensure the ...,JenPetterson,Planet 🌍 Earth,congress must act now to ensure the safety of...
4,None,RT @francesleach_: Cuban doctors arriving into...,afro_rriquena,"Manhattan, NY",cuban doctors arriving into italy to suppo th...


In [27]:
df_tweets.to_csv('../Data/Tweets_scraped_2000_23rd_CORONA_COVID_FTC.csv')